In [209]:
import pandas as pd
import numpy as np

In [16]:
train_data_path = "Data/webis-clickbait-22/train.jsonl"

# Predict

In [233]:
from simpletransformers.classification import ClassificationModel
import torch


# load file
def load_input(df):
    if type(df) != pd.DataFrame:
        df = pd.read_json(df, lines=True)
    
    ret = []
    for _, i in df.iterrows():
        ret += [{'text': ' '.join(i['postText']) + ' - ' + i['targetTitle'] + ' ' + ' '.join(i['targetParagraphs']), 'uuid': i['uuid']}]
    
    return pd.DataFrame(ret)



def use_cuda():
    return torch.cuda.is_available() and torch.cuda.device_count() > 0


def predict(df):
    # df = load_input(df)
    labels = ['phrase', 'passage', 'multi']
    model = ClassificationModel('deberta',"webis/spoiler-type-classification", use_cuda=use_cuda())

    uuids = list(df['uuid'])
    texts = list(df['text'])
    predictions = model.predict(texts)[1]
    
    for i in range(len(df)):
        yield {'uuid': uuids[i], 'spoilerType': labels[np.argmax(predictions[i])]}


def run_baseline(input_file, output_file):
    with open(output_file, 'w') as out:
        for prediction in predict(input_file):
            out.write(json.dumps(prediction) + '\n')

In [234]:
data = load_input(train_data_path)
data.head()

,text,uuid
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",0af11f6b-c889-4520-9372-66ba25cb7657
1,NASA sets date for full recovery of ozone hole...,b1a1f63d-8853-4a11-89e8-6b2952a393ec
2,This is what makes employees happy -- and it's...,008b7b19-0445-4e16-8f9e-075b73f80ca4
3,Passion is overrated — 7 work habits you need ...,31ecf93c-3e21-4c80-949b-aa549a046b93
4,The perfect way to cook rice so that it's perf...,31b108a3-c828-421a-a4b9-cf651e9ac859


In [235]:
for prediction in predict(data):
    print(prediction)
    break

OSError: webis/spoiler-type-classification does not appear to have a file named config.json. Checkout 'https://huggingface.co/webis/spoiler-type-classification/main' for available files.